In [ ]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [ ]:
# Import the relevant modules
import sousvide.synthesize.rollout_generator as rg
import sousvide.synthesize.observation_generator as og
import sousvide.instruct.train_policy as tp
import sousvide.visualize.plot_synthesize as ps
import sousvide.visualize.plot_learning as pl
import sousvide.flight.deploy_figs as df

In [ ]:
cohort = "experimental"             # cohort name

data_method = "data_beta"           # method name
eval_method = "eval_nominal"        # method name

scene = "mid_gate"                  # scene name

courses = [                         # course names
    "robustness_track",
    ]   

roster = [
    "Maverick",                     # sifu_testbed
    "Rooster",                      # sift_testbed
    ]

Neps_sw = 10                        # step-wise number of epochs
Neps_hN = 200                        # histNet number of epochs 
Neps_cN = 200                       # commNet number of epochs

In [ ]:
# Generate Rollouts
rg.generate_rollout_data(cohort,courses,scene,data_method)

# Review the Rollout Data
ps.plot_rollout_data(cohort)


In [ ]:
# Generate initial observation data sets
og.generate_observation_data(cohort,roster)

In [ ]:
# First Stage Training
tp.train_roster(cohort,roster,"histNet",50,regen=False)

for i in range(10):
    tp.train_roster(cohort,roster,"commNet",20,regen=True,use_deploy=scene)

pl.plot_losses(cohort,roster,"histNet")
pl.plot_losses(cohort,roster,"commNet")


In [ ]:
# Second Stage Training
tp.train_roster(cohort,roster,"histNet",50,regen=False)

for i in range(10):
    tp.train_roster(cohort,roster,"commNet",20,regen=True,use_deploy=scene)

pl.plot_losses(cohort,roster,"histNet")
pl.plot_losses(cohort,roster,"commNet")

In [ ]:
# Simulate in FiGS
for course in courses:
    df.deploy_roster(cohort,course,scene,eval_method,roster)
    pl.plot_deployments(cohort,course,roster,plot_show=True)